In [1]:
import pandas as pd

In [3]:
df = pd.read_parquet('../do_not_commit/Datasets/Persist_Operating_System_DATA.pq')

In [4]:
features = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE']).agg(
    {
        'InstallDate00':'max',
        'LastBootUpTime00':'min',
        'TotalVirtualMemorySize00':'max'
    }
).reset_index()
features

,MachineID,RWB_EFFECTIVE_DATE,InstallDate00,LastBootUpTime00,TotalVirtualMemorySize00
0,16783564,2023-01-19,2021-04-02 14:25:05,2023-01-16 02:00:46,16895
1,16783564,2023-01-20,2021-04-02 14:25:05,2023-01-16 02:00:46,16895
2,16783564,2023-01-21,2021-04-02 14:25:05,2023-01-16 02:00:46,16895
3,16783564,2023-01-22,2021-04-02 14:25:05,2023-01-16 02:00:46,16895
4,16783564,2023-01-23,2021-04-02 14:25:05,2023-01-23 02:02:49,16895
...,...,...,...,...,...
1078111,16819026,2023-07-03,2021-06-25 01:06:29,2023-07-03 02:02:14,33569
1078112,16819026,2023-07-04,2021-06-25 01:06:29,2023-07-04 02:00:43,33569
1078113,16819027,2023-07-03,2021-03-26 15:25:44,2023-07-03 02:00:49,67376
1078114,16819027,2023-07-04,2021-03-26 15:25:44,2023-07-03 02:00:49,67376


In [15]:
from datetime import datetime
from datetime import date
import numpy as np

def calculate_age_in_days(target_date_str, in_date):

    # Convert target_date_str to a Python datetime object
    target_date = datetime.strptime(target_date_str, '%Y-%m-%d').date()
    in_date = datetime.strptime(in_date, '%Y-%m-%d').date()

    # Calculate the age in days
    age_in_days = (in_date - target_date).days

    return age_in_days


def create_program_features(in_df):

    # Change effective date to type date
    in_df['InstallDate00'] = pd.to_datetime(in_df['InstallDate00']).dt.strftime('%Y-%m-%d')

    # Get age in days
    todaydate = date(2023, 7, 4)
    in_df['InstallAge'] = in_df['InstallDate00'].apply(lambda x: calculate_age_in_days(x, todaydate)
    if pd.notnull(x) else np.nan)
    return in_df


def create_last_boot_features(in_df):

    # Change effective date to type date
    in_df['LastBootUpTime00'] = pd.to_datetime(in_df['LastBootUpTime00']).dt.strftime('%Y-%m-%d')
    in_df['RWB_EFFECTIVE_DATE'] = in_df['RWB_EFFECTIVE_DATE'].astype(str)

    # Get age in days
    in_df['LastBootAge'] = in_df.apply(lambda x: calculate_age_in_days(x['LastBootUpTime00'], x['RWB_EFFECTIVE_DATE'])
    if pd.notnull(x['LastBootUpTime00']) else np.nan, axis=1)
    return in_df

In [11]:
df = create_program_features(features)

In [16]:
df = create_last_boot_features(df)

In [17]:
df.sample(20)

,MachineID,RWB_EFFECTIVE_DATE,InstallDate00,LastBootUpTime00,TotalVirtualMemorySize00,InstallAge,LastBootAge
742428,16814218,2023-02-25,2022-07-12,2023-02-23,36758,357.0,2.0
379057,16808266,2023-03-13,2021-09-27,2023-03-13,32490,645.0,0.0
848355,16815005,2023-01-26,2021-09-14,2023-01-25,32490,658.0,1.0
187152,16802767,2023-04-12,2020-12-17,2023-04-11,33598,929.0,1.0
253124,16804975,2023-06-30,2021-05-18,2023-06-25,32431,777.0,5.0
493140,16810244,2023-02-12,2022-03-30,2023-02-11,32494,461.0,1.0
248900,16804874,2023-04-20,2021-05-10,2023-04-19,32435,785.0,1.0
144261,16800757,2023-03-14,2021-06-17,2023-03-13,33569,747.0,1.0
512862,16810697,2023-06-03,2022-03-07,2023-06-03,32490,484.0,0.0
755988,16814316,2023-02-01,2021-11-01,2023-01-26,32490,610.0,6.0


In [18]:
df.to_parquet('../do_not_commit/FeatureDatasets/operating_system_features.pq')


# DEPRECATED.



# os_install_date_age


In [4]:
# Age of the operating system calculated from the os_install_age.
# .groupby([MachineID, RWB_effective_date]) and use the .max() value of 'os_install_age'.

# RJ 08/01/23: 'os_install_age' doesn't exist. Using 'InstallDate00'.
# os_install_age = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['InstallDate00'].agg('max').reset_index()
# os_install_age.to_parquet('../do_not_commit/FeatureDatasets/os_install_age.pq')
# os_install_age


# last_boot_up_time


In [5]:
# Number of days since the machine was last booted.
# Can calculate this from the Operating_System_DATA table using the LastBootUpTime0 attribute.
# .groupby([MachineID, RWB_effective_date]) and use the .min() value of 'LastBootUpTime00'.

# last_boot = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['LastBootUpTime00'].agg('min').reset_index()
# last_boot.to_parquet('../do_not_commit/FeatureDatasets/last_boot_up_time.pq')
# last_boot


# os_total_virtual_memory_size


In [6]:
# Total virtual operating system memory size.
# .groupby([MachineId, RWB_effective_date]) and use the .max() value of 'TotalVirtualMemorySize00'.

# os_total_virtual_memory_size = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['TotalVirtualMemorySize00'].agg('max').reset_index()
# os_total_virtual_memory_size.to_parquet('../do_not_commit/FeatureDatasets/os_total_virtual_memory_size.pq')
# os_total_virtual_memory_size